In [2]:
!git clone https://github.com/cleverhans-lab/cleverhans.git
%cd cleverhans
!pip install -r "/content/cleverhans/requirements/requirements.txt"
!pip install -e .
import cleverhans
print(cleverhans.__version__)


fatal: destination path 'cleverhans' already exists and is not an empty directory.
/content/cleverhans/cleverhans
Obtaining file:///content/cleverhans/cleverhans
ERROR: file:///content/cleverhans/cleverhans does not appear to be a Python project: neither 'setup.py' nor 'pyproject.toml' found.
4.0.0-c85ef5e77db2b4811ce88bfdc5a82aa3


In [1]:
# Install the latest stable version of CleverHans from PyPI
!pip install cleverhans

# Optionally, install the bleeding edge version from the GitHub repository
!pip install git+https://github.com/cleverhans-lab/cleverhans.git#egg=cleverhans

# For development installation (editable mode):
# First, clone your fork of the repository (replace <your-org> with your GitHub organization or username)
!git clone https://github.com/<your-org>/cleverhans

# Navigate into the cloned directory
%cd cleverhans

# Install in editable mode
!pip install -e .


  Cloning https://github.com/cleverhans-lab/cleverhans.git to /tmp/pip-install-i6ylzanu/cleverhans_64c6cb41530d4785955f3fd9a2b48e30
  Running command git clone --filter=blob:none --quiet https://github.com/cleverhans-lab/cleverhans.git /tmp/pip-install-i6ylzanu/cleverhans_64c6cb41530d4785955f3fd9a2b48e30
  Resolved https://github.com/cleverhans-lab/cleverhans.git to commit 574efc1d2f5c7e102c78cf0e937654e847267522
  Preparing metadata (setup.py) ... done
/bin/bash: line 1: your-org: No such file or directory
/content/cleverhans
Obtaining file:///content/cleverhans
  Preparing metadata (setup.py) ... done
  Attempting uninstall: cleverhans
    Found existing installation: cleverhans 4.0.0
    Uninstalling cleverhans-4.0.0:
      Successfully uninstalled cleverhans-4.0.0
  Running setup.py develop for cleverhans


In [12]:
import numpy as np
import tensorflow as tf
import tensorflow_datasets as tfds
from easydict import EasyDict
from tensorflow.keras import Model
from tensorflow.keras.layers import Dense, Dropout, Flatten, Conv2D

from cleverhans.tf2.attacks.projected_gradient_descent import projected_gradient_descent
from cleverhans.tf2.attacks.fast_gradient_method import fast_gradient_method

# Remove absl.flags dependency (use default Python arguments or globals)

class Net(Model):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = Conv2D(64, 8, strides=(2, 2), activation="relu", padding="same")
        self.conv2 = Conv2D(128, 6, strides=(2, 2), activation="relu", padding="valid")
        self.conv3 = Conv2D(128, 5, strides=(1, 1), activation="relu", padding="valid")
        self.dropout = Dropout(0.25)
        self.flatten = Flatten()
        self.dense1 = Dense(128, activation="relu")
        self.dense2 = Dense(10)

    def call(self, x):
        x = self.conv1(x)
        x = self.conv2(x)
        x = self.conv3(x)
        x = self.dropout(x)
        x = self.flatten(x)
        x = self.dense1(x)
        return self.dense2(x)


def ld_mnist():
    """Load training and test data."""

    def convert_types(image, label):
        image = tf.cast(image, tf.float32)
        image /= 255
        return image, label

    # Load MNIST dataset from TensorFlow Datasets
    dataset, info = tfds.load(
        "mnist", with_info=True, as_supervised=True, data_dir='/tmp/tensorflow_datasets'
    )
    mnist_train, mnist_test = dataset["train"], dataset["test"]
    mnist_train = mnist_train.map(convert_types).shuffle(10000).batch(128)
    mnist_test = mnist_test.map(convert_types).batch(128)
    return EasyDict(train=mnist_train, test=mnist_test)


def main(nb_epochs=8, eps=0.3, adv_train=False):
    # Load training and test data
    data = ld_mnist()
    model = Net()
    loss_object = tf.losses.SparseCategoricalCrossentropy(from_logits=True)
    optimizer = tf.optimizers.Adam(learning_rate=0.001)

    # Metrics to track the different accuracies.
    train_loss = tf.metrics.Mean(name="train_loss")
    test_acc_clean = tf.metrics.SparseCategoricalAccuracy()
    test_acc_fgsm = tf.metrics.SparseCategoricalAccuracy()
    test_acc_pgd = tf.metrics.SparseCategoricalAccuracy()

    @tf.function
    def train_step(x, y):
        with tf.GradientTape() as tape:
            predictions = model(x)
            loss = loss_object(y, predictions)
        gradients = tape.gradient(loss, model.trainable_variables)
        optimizer.apply_gradients(zip(gradients, model.trainable_variables))
        train_loss(loss)

    # Train model with adversarial training
    for epoch in range(nb_epochs):
        progress_bar_train = tf.keras.utils.Progbar(60000)
        for (x, y) in data.train:
            if adv_train:
                # Replace clean example with adversarial example for adversarial training
                x = projected_gradient_descent(model, x, eps, 0.01, 40, np.inf)
            train_step(x, y)
            progress_bar_train.add(x.shape[0], values=[("loss", train_loss.result())])

    # Evaluate on clean and adversarial data
    progress_bar_test = tf.keras.utils.Progbar(10000)
    for x, y in data.test:
        y_pred = model(x)
        test_acc_clean(y, y_pred)

        x_fgm = fast_gradient_method(model, x, eps, np.inf)
        y_pred_fgm = model(x_fgm)
        test_acc_fgsm(y, y_pred_fgm)

        x_pgd = projected_gradient_descent(model, x, eps, 0.01, 40, np.inf)
        y_pred_pgd = model(x_pgd)
        test_acc_pgd(y, y_pred_pgd)

        progress_bar_test.add(x.shape[0])

    print(
        "test acc on clean examples (%): {:.3f}".format(test_acc_clean.result() * 100)
    )
    print(
        "test acc on FGM adversarial examples (%): {:.3f}".format(
            test_acc_fgsm.result() * 100
        )
    )
    print(
        "test acc on PGD adversarial examples (%): {:.3f}".format(
            test_acc_pgd.result() * 100
        )
    )


# Modify the default values directly and run main()
if __name__ == "__main__":
    # Set default values for arguments
    nb_epochs = 8
    eps = 0.3
    adv_train = False  # Set to True for adversarial training

    main(nb_epochs=nb_epochs, eps=eps, adv_train=adv_train)


Dl Completed...:   0%|          | 0/5 [00:00<?, ? file/s]

Dataset mnist downloaded and prepared to /tmp/tensorflow_datasets/mnist/3.0.1. Subsequent calls will reuse this data.
60000/60000 ━━━━━━━━━━━━━━━━━━━━ 143s 2ms/step - loss: 0.4253
60000/60000 ━━━━━━━━━━━━━━━━━━━━ 135s 2ms/step - loss: 0.1487
60000/60000 ━━━━━━━━━━━━━━━━━━━━ 135s 2ms/step - loss: 0.1051
60000/60000 ━━━━━━━━━━━━━━━━━━━━ 139s 2ms/step - loss: 0.0832
60000/60000 ━━━━━━━━━━━━━━━━━━━━ 135s 2ms/step - loss: 0.0696
60000/60000 ━━━━━━━━━━━━━━━━━━━━ 134s 2ms/step - loss: 0.0599
60000/60000 ━━━━━━━━━━━━━━━━━━━━ 135s 2ms/step - loss: 0.0529
60000/60000 ━━━━━━━━━━━━━━━━━━━━ 134s 2ms/step - loss: 0.0475
10000/10000 ━━━━━━━━━━━━━━━━━━━━ 700s 70ms/step
test acc on clean examples (%): 98.900
test acc on FGM adversarial examples (%): 14.760
test acc on PGD adversarial examples (%): 1.450
